# df_points with UNIX Timestamp #
## Creates df_points.csv with all points and idletimes #
## Checks the valid geo location

### ca 1h for 1 year

In [1]:
def importdata():

    # show all rows
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)

    # import Data
    df01 = pd.read_csv('/Users/jonweske/devment/BA_nextbike/data/lendings_2019.csv')


    df01 = df01[['bike_id', 'start_time', 'end_time', 'start_lat', 'start_lng', 'end_lat', 'end_lng']]

    # cut
    #df01 = df01.truncate(after=10000)
    return df01

In [2]:
def dicOutOfDf(df01):
    import pandas as pd
    from tqdm import tqdm
    # Dict out of DataFrames of Trips per Bike

    # init DFs for every bike
    myDFs = {}

    for ind in tqdm(df01.index):
        if df01['bike_id'][ind] not in myDFs:
            myDFs[df01['bike_id'][ind]] = pd.DataFrame(
                columns=['bike_id', 'start_time', 'end_time', 'start_lat', 'start_lng', 'end_lat', 'end_lng'])

    return myDFs

In [7]:
def addRentalsToDf(df01,myDFs):
    from tqdm import tqdm
    # Add all rentals to their bike_id DF
    for ind in tqdm(df01.index):
        if df01['bike_id'][ind] in myDFs:
            myDFs[df01['bike_id'][ind]].loc[df01.index[ind]] = df01.iloc[ind]
    return myDFs

In [8]:
def sortDFs(myDFs):
    # sort every DF
    for df in myDFs.values():
        df[['bike_id', 'start_time','end_time']] = df[['bike_id', 'start_time','end_time']].astype(int)
        df[['start_lat','start_lng', 'end_lat', 'end_lng']] = df[['start_lat','start_lng', 'end_lat', 'end_lng']].astype(float)
        df.sort_values(by=['start_time'], inplace=True)
        df.reset_index(drop=True, inplace=True)
    return myDFs

In [14]:
def dfPoints(myDFs,radius):
    # DF of all points with stoodtime
    inval = 0
    val = 0 
    
    df_points = pd.DataFrame(columns=['bike_id', 'lat', 'lng', 'timestood', 'time_start', 'time_end'])
    df_points[['bike_id', 'timestood', 'time_start', 'time_end']] = df_points[['bike_id', 'timestood', 'time_start', 'time_end']].astype(int)
    df_points[['lng', 'lat']] = df_points[['lng', 'lat']].astype(float)
    
    for df in tqdm(myDFs.values()):
        for ind in df.index[1:]:
            #in SECOUNDS
            timestood = (df['start_time'][ind] - df['end_time'][ind - 1]).astype(int)
            distance = qick_distance(df['end_lat'][ind-1], df['end_lng'][ind-1], df['start_lat'][ind], df['start_lng'][ind])
            if distance > radius:
                inval +=1
                continue
            val +=1
            dict = {'bike_id': df['bike_id'][ind], 'lng': df['start_lng'][ind], 'lat': df['start_lat'][ind],
                    'timestood': timestood, 'time_start': df['end_time'][ind - 1], 'time_end': df['start_time'][ind]}
            df_points = df_points.append(dict, ignore_index=True)
    print(f"invalides: {inval} from {val} thats {inval/val * 100} %")
    return df_points

In [10]:
#old
def diff(Elat, Elng, Slat, Slng):
    xdiff = abs(round(Elat,8)-round(Slat,8)) * 100000
    ydiff = abs(round(Elng,8)-round(Slng,8)) * 100000
    #print(f"{xdiff} und {ydiff}")
    return xdiff,ydiff

In [ ]:
# returns distance in meter
# source: https://www.it-swarm.com.de/de/python/wie-kann-ich-die-entfernung-zwischen-zwei-punkten-breitengrad-laengengrad-schnell-schaetzen/1072488907/
def qick_distance(Lat1, Long1, Lat2, Long2):
    x = Lat2 - Lat1
    y = (Long2 - Long1)*cos((Lat2 + Lat1)*0.00872664626)  
    return 111.138*sqrt(x*x+y*y)*1000

In [11]:
def keplerMap(df_points):
    heatmap = KeplerGl()
    heatmap.add_data(data=df_points, name='points')
    heatmap.save_to_html(file_name='heatmap_test.html')

# Main

In [12]:
from keplergl import KeplerGl
import pandas as pd
pd.options.mode.chained_assignment = None
from tqdm import tqdm
from geopy.distance import geodesic 
df = pd.DataFrame()


df = importdata()
myDfs = dicOutOfDf(df)
myDfs = addRentalsToDf(df, myDfs)
myDfs = sortDFs(myDfs)


# save CSV and Map
#df_points.to_csv('df_points.csv', index=False)
#keplerMap(df_points)


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 604598/604598 [12:14<00:00, 823.38it/s]


## Create DF_points 
### without ~9.573 % invalid points

In [15]:
# create DF_POints
radius = 20
df_points = dfPoints(myDfs,radius)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1210/1210 [36:28<00:00,  1.81s/it]

invalides: 52718 from 550670 thats 9.573428732271596 %


## Save MyDfs Map to file 

In [18]:
import pickle
with open('saved_myDfs.pkl', 'wb') as f:
    pickle.dump(myDfs, f)

## Save df_points_valid_geov01.csv

In [17]:
df_points.to_csv("df_points_valid_geo_v01.csv",index=False)

In [10]:
Alat = 51.300405
Alng = 12.311043
Blat = 51.301405
Blng = 12.312043

In [11]:
def diff2(Elat, Elng, Slat, Slng):
    xdiff = abs(round(Elat,8)-round(Slat,8)) * 100000
    ydiff = abs(round(Elng,8)-round(Slng,8)) * 100000
    #print(f"{xdiff} und {ydiff}")
    return xdiff,ydiff

In [15]:
from math import radians, cos, sin, asin, sqrt
print(qick_distance(Alat,Alng,Blat,Blng))

131.07279326442534
